In [1]:
import statsmodels.api as sm
import pandas as pd
import numpy as np
import datetime
import matplotlib.pyplot as plt
from linearmodels import PanelOLS
from patsy import dmatrices
from statsmodels.stats.outliers_influence import variance_inflation_factor
from sklearn.model_selection import train_test_split
from sklearn.linear_model import Ridge
from yellowbrick.datasets import load_concrete
from yellowbrick.regressor import ResidualsPlot
from scipy import stats

In [2]:
joint = pd.read_excel('joint_snap.xlsx')

In [4]:
#convert the month column into correct form
joint['Month'] = pd.to_datetime(joint['Month'])
age = (joint['Month'] - joint['Inception Date'])/np.timedelta64(1,'M')
age = [int(i) for i in age]
joint['Age'] = age

#relative_flow is the size divided by flow value
flow = joint["Fund Flow"]
size = joint['Fund Size']
relative_flow = flow/size
joint['relative_flow'] = relative_flow

joint_new = joint.rename(columns={"Fund Sustainability": "esg_score"})

In [5]:
#build a new column, ESG_Score(t-1)
esg_ex = joint_new['esg_score']

esg_mod = [np.nan]*len(esg_ex)

for i in range(3276):
    for j in range(0,91):
         esg_mod[92*i+j+1] = esg_ex[92*i+j]
            
joint_new['esg_mod'] = esg_mod

In [38]:
#build a new column, in which the relative_flow is the value of last month
capm_org = joint_new['performance_Four_Factor']
capm_nxt = [np.nan]*len(capm_org)

verschiebung = 1

for i in range(3276):
    for j in range(verschiebung,92):
         capm_nxt[92*i+j-verschiebung] = capm_org[92*i+j]
            
joint_new['capm_Performance_mod'] = capm_nxt

In [39]:
#catagorize the month - 2012/01 to 2019/08, totoally 92 months
month = pd.Categorical(joint_new.Month)

#use the 'Month' to divide the dataset, the index is Fund ID
df = joint_new.set_index(['FundId', 'Month'])
df['Month'] = month

In [40]:
df

Fund Size     Fund Flow  esg_score  \
FundId     Month                                               
FS00008SZD 2012-01-01  2.609599e+08  4.379443e+07        NaN   
           2012-02-01  2.988403e+08  2.572761e+07        NaN   
           2012-03-01  3.071028e+08  1.558167e+06        NaN   
           2012-04-01  3.440745e+08  3.753147e+07        NaN   
           2012-05-01  3.771187e+08  4.877692e+07        NaN   
...                             ...           ...        ...   
FSUSA09WNF 2019-04-01  8.137770e+06 -1.487095e+03        NaN   
           2019-05-01  7.979041e+06  8.084123e+02        NaN   
           2019-06-01  8.651099e+06  2.060187e+03      40.91   
           2019-07-01  8.737163e+06 -2.267210e+03        NaN   
           2019-08-01  8.255823e+06  2.131805e+03        NaN   

                       capm_Performance  performance_Three_Factor  \
FundId     Month                                                    
FS00008SZD 2012-01-01          1.507391                  3.652442   
           2012-02-01          1.544497                  1.220171   
           2012-03-01          4.025839                  0.078026   
           2012-04-01          2.647697                  0.989935   
           2012-05-01         -0.066138                 -0.807304   
...                                 ...                       ...   
FSUSA09WNF 2019-04-01         -3.067829                 -3.546688   
           2019-05-01          3.666930                  4.017213   
           2019-06-01          4.207674                  3.772864   
           2019-07-01          1.134835                  4.536452   
           2019-08-01         -5.479327                 -3.326145   

                       performance_Four_Factor Inception Date  \
FundId     Month                                                
FS00008SZD 2012-01-01                 1.841484     2009-01-28   
           2012-02-01                 1.155288     2009-01-28   
           2012-03-01                 0.963080     2009-01-28   
           2012-04-01                 1.171659     2009-01-28   
           2012-05-01                -0.354775     2009-01-28   
...                                        ...            ...   
FSUSA09WNF 2019-04-01                -4.150616     2008-11-11   
           2019-05-01                 5.035920     2008-11-11   
           2019-06-01                 3.199166     2008-11-11   
           2019-07-01                 4.007588     2008-11-11   
           2019-08-01                -2.716709     2008-11-11   

                      Equity Style Box (Long)  Age  size_log  relative_flow  \
FundId     Month                                                              
FS00008SZD 2012-01-01            Large Growth   35  8.416574       0.167821   
           2012-02-01            Large Growth   36  8.475439       0.086092   
           2012-03-01            Large Growth   37  8.487284       0.005074   
           2012-04-01            Large Growth   38  8.536653       0.109079   
           2012-05-01            Large Growth   39  8.576478       0.129341   
...                                       ...  ...       ...            ...   
FSUSA09WNF 2019-04-01             Large Blend  124  6.910505      -0.000183   
           2019-05-01             Large Blend  125  6.901951       0.000101   
           2019-06-01             Large Blend  126  6.937071       0.000238   
           2019-07-01             Large Blend  127  6.941370      -0.000259   
           2019-08-01             Large Blend  128  6.916760       0.000258   

                       esg_mod  capm_Performance_mod      Month  
FundId     Month                                                 
FS00008SZD 2012-01-01      NaN              1.155288 2012-01-01  
           2012-02-01      NaN              0.963080 2012-02-01  
           2012-03-01      NaN              1.171659 2012-03-01  
           2012-04-01      NaN             -0.354775 2012-04-01  
           2012-05-01    

In [45]:
#capm Performance summary
exog_vars = ['esg_score','esg_mod']
exog = sm.add_constant(df[exog_vars])

mod = PanelOLS(df.capm_Performance_mod, exog, entity_effects=True, time_effects=True)

res = mod.fit()

In [46]:
print(res)

                           PanelOLS Estimation Summary                            
Dep. Variable:     capm_Performance_mod   R-squared:                     9.676e-05
Estimator:                     PanelOLS   R-squared (Between):             -0.0067
No. Observations:                145943   R-squared (Within):               0.0005
Date:                  Wed, Apr 08 2020   R-squared (Overall):          -2.702e-05
Time:                          09:49:40   Log-likelihood                -3.123e+05
Cov. Estimator:              Unadjusted                                           
                                          F-statistic:                      6.9153
Entities:                          2928   P-value                           0.0010
Avg Obs:                         49.844   Distribution:                F(2,142925)
Min Obs:                         1.0000                                           
Max Obs:                         89.000   F-statistic (robust):             6.9153
    